# Preparing the KEN datasets

In this notebook I will be looking at the raw versions of the KEN datasets, then 
follow the pre-processing steps used for the KEN baseline. Note that I am using 
YAGO 2022, so that might introduce discrepancies between this and the original
version. 

Datasets:
- [The Movies Dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset)
- [7+ Million Company Dataset](https://www.kaggle.com/datasets/peopledatalabssf/free-7-million-company-dataset)
- [US Accidents (2016 - 2021)](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents)

In [33]:
import polars as pl
import pandas as pd
from pathlib import Path
import src.yago.utils as utils


In [10]:
cd ~/work/prepare-data-lakes

/home/soda/rcappuzz/work/prepare-data-lakes


In [4]:
data_dir = Path("data/kaggle")

Loading YAGO fact triplets to drop entities not found in the KB.

In [ ]:
yago_path = Path("/storage/store3/work/jstojano/yago3/")
facts_path = Path(yago_path, "facts_parquet/yago_updated_2022_part2")
fname = "yagoFacts"
yagofacts_path = Path(facts_path, f"{fname}.tsv.parquet")
yagofacts = utils.import_from_yago(yagofacts_path, engine="polars")

# US Accidents dataset

Archive `us-accidents.zip` contains the file `US_Accidents_Dec21_updated.csv`, 
which I renamed manually to `us-accidents.csv` for simplicity. 

I also had to copy the file `datasets/us_accidents/state_codes.csv` from the 
KEN repository for some of the steps. 

In [15]:
dataset_dir = Path(data_dir, "us-accidents")

In [11]:
df = pl.read_csv(Path(dataset_dir, "us-accidents.csv"))
df = df.rename({"State": "Code"})

In [16]:
state_codes_path =  Path(dataset_dir,"state_codes.csv")
state_codes = pl.read_csv(state_codes_path)
df = df.join(
    state_codes, on="Code"
)

Adding a new column, `col_to_embed`, that formats the city and state name to have
the same format that is found in YAGO. 

In [30]:
df = df.with_columns(
    ("<" + pl.col("City") + ",_"+ pl.col("State") + ">").alias("col_to_embed")
)

Filtering out the rows not found in `yagofacts["subject"]`.

In [41]:
df_filtered=df.lazy().filter(
    pl.col("col_to_embed").is_in(
        yagofacts["subject"]
    )
).collect()

Completing the preparation by grouping the number of accidents by city, then applying the log10 to the count. This is 
the target used in KEN. 


In [63]:
df_counts = df_filtered.groupby(
    [
        "col_to_embed", "City", "Code"
    ]
    ).count()

In [64]:
df_counts_prepared = df_counts.with_columns(
    (df_counts["City"] + ", " + df_counts["Code"]).alias("raw_entities") 
).select(
    [
        pl.col("raw_entities"),
        pl.col("col_to_embed"),
        pl.col("count").alias("target").log10()
    ]
).sort("raw_entities")

In [65]:
df_counts_prepared

raw_entities,col_to_embed,target
str,str,f64
"""Abbeville, AL""","""<Abbeville,_Al…",0.778151
"""Abbeville, LA""","""<Abbeville,_Lo…",0.0
"""Abbotsford, WI…","""<Abbotsford,_W…",0.954243
"""Abbottstown, P…","""<Abbottstown,_…",1.39794
"""Aberdeen, ID""","""<Aberdeen,_Ida…",0.0
"""Aberdeen, MD""","""<Aberdeen,_Mar…",2.71433
"""Aberdeen, MS""","""<Aberdeen,_Mis…",0.477121
"""Aberdeen, OH""","""<Aberdeen,_Ohi…",0.0
"""Aberdeen, WA""","""<Aberdeen,_Was…",1.556303


In [66]:
df_counts_prepared.write_parquet(Path(dataset_dir, "us-accidents-counts.parquet"))

Reading original to see if the two versions look similar. 

In [67]:
path_original=Path("/storage/store3/work/acvetkov/gitlab/KEN/experiments/datasets/us_accidents/counts.parquet")
df_counts_og = pl.read_parquet(path_original)
df_counts_og

raw_entities,col_to_embed,target
str,str,f64
"""Aaronsburg, PA…","""<Aaronsburg,_P…",0.30103
"""Abbeville, LA""","""<Abbeville,_Lo…",0.0
"""Abbotsford, WI…","""<Abbotsford,_W…",0.954243
"""Abbottstown, P…","""<Abbottstown,_…",1.041393
"""Aberdeen, MD""","""<Aberdeen,_Mar…",2.396199
"""Aberdeen, MS""","""<Aberdeen,_Mis…",0.477121
"""Aberdeen, OH""","""<Aberdeen,_Ohi…",0.0
"""Aberdeen, WA""","""<Aberdeen,_Was…",1.342423
"""Abernathy, TX""","""<Abernathy,_Te…",0.0


As expected, the number of entities in the original file is smaller. 

# Company Employees Dataset

In [69]:
dataset_dir = Path(data_dir, "company-employees")

In [70]:
df = pl.read_csv(Path(dataset_dir, "companies_sorted.csv"))
df_selected = df.filter(
    pl.col("current employee estimate") >= 1000
)

I am using a slightly different euristic compared to what Alexis was using. 

Adding a new column to `yagofacts` with lowercased subjects. 

In [ ]:
yagofacts = yagofacts.with_columns(
    pl.col("subject").str.to_lowercase().alias("subject_formatted")
)

In [116]:
df_filtered=df_selected.lazy().with_columns(
    ("<" + pl.col("name").str.to_lowercase().str.replace(" ", "_") + ">").alias("formatted_name")
).filter(
    pl.col("formatted_name").is_in(yagofacts["subject_formatted"])
).collect()

Here I am preparing a mapping between the name in the original dataset and the match found in YAGO.
Note that there is a relatively low recall, though it is higher than what is used in the original. 

In [117]:
mapping_name_subject = df_filtered.lazy().join(
    yagofacts.lazy(),
    left_on="formatted_name",
    right_on="subject_formatted"
).select(
    [
        pl.col("name"),
        pl.col("formatted_name"),
        pl.col("subject")
    ]
).unique().collect()


Joining on with the mapping on `formatted_name` to guarantee that col `col_to_embed` uses the same format (and 
capitalization) used in YAGO. 

In [120]:
df_target = df_filtered.join(
    mapping_name_subject, on="formatted_name"
).select(
    [
        pl.col("name").alias("raw_entities"),
        pl.col("subject").alias("col_to_embed"),
        pl.col("current employee estimate").alias("target").log10()
    ]
)

In [121]:
df_target

raw_entities,col_to_embed,target
str,str,f64
"""ibm""","""<IBM>""",5.437825
"""accenture""","""<Accenture>""",5.280326
"""hewlett-packar…","""<Hewlett-Packa…",5.107047
"""walmart""","""<Walmart>""",5.081898
"""microsoft""","""<Microsoft>""",5.065191
"""at&t""","""<AT&T>""",5.061407
"""wells fargo""","""<Wells_Fargo>""",5.039541
"""infosys""","""<Infosys>""",5.020162
"""deloitte""","""<Deloitte>""",5.017501


In [122]:
path_original=Path("/storage/store3/work/acvetkov/gitlab/KEN/experiments/datasets/company_employees/target.parquet")
df_target_og = pl.read_parquet(path_original)
df_target_og

raw_entities,col_to_embed,target
str,str,f64
"""accenture""","""<Accenture>""",5.280326
"""walmart""","""<Walmart>""",5.081898
"""microsoft""","""<Microsoft>""",5.065191
"""infosys""","""<Infosys>""",5.020162
"""deloitte""","""<Deloitte>""",5.017501
"""nokia""","""<Nokia>""",4.925967
"""capgemini""","""<Capgemini>""",4.925204
"""google""","""<Google>""",4.875692
"""ericsson""","""<Ericsson>""",4.830537


Here I am checking which values in `col_to_embed` are not found in `yagofacts["subject"]`. These companies are missing 
because the name of the company is not the same in the original dataset and in YAGO. 

In [123]:
df_target_og.filter(
    ~pl.col("col_to_embed").is_in(yagofacts["subject"])
)

raw_entities,col_to_embed,target
str,str,f64
"""allstate""","""<Allstate>""",4.498173
"""raytheon""","""<Raytheon>""",4.375353
"""thales""","""<Thales>""",4.321868
"""herbalife""","""<Herbalife>""",4.289433
"""flextronics""","""<Flextronics>""",4.279644
"""stryker""","""<Stryker>""",4.19201
"""adecco""","""<Adecco>""",4.160799
"""altran""","""<Altran>""",4.090187
"""statoil""","""<Statoil>""",4.081563


# The Movies Dataset

In [126]:
dataset_dir = Path(data_dir, "the-movies-dataset")
df = pl.read_csv(Path(dataset_dir, "movies_metadata.csv"), infer_schema_length=0)


In [153]:
# Filtering
df_filtered = df.filter(
    pl.col("revenue").cast(int) > 0
).with_columns(
    (pl.col("release_date").str.slice(0, 4)).alias("release_date")
)

In [189]:
# Prepare 3 different mappings, to try and cover as many cases as possible
mapping_to_yago = df_filtered.select([pl.col("title"), pl.col("release_date"), pl.col("revenue")])


In [190]:
mapping_to_yago=mapping_to_yago.with_columns(
    [
        ("<" + pl.col("title").str.replace(" ", "_") + ">").alias("title_format_1"),
        ("<" + pl.col("title").str.replace(" ", "_") + "_(film)>").alias("title_format_2"),
        ("<" + pl.col("title").str.replace(" ", "_") +  "_(" + pl.col("release_date") + "_film)>").alias("title_format_3"),
        pl.Series(list(range(len(mapping_to_yago)))).alias("index")
    ]
)

In [214]:
tgt_indices = []
for jj in [3,2,1]:
    g1 = mapping_to_yago.filter(
        (pl.col(f"title_format_{jj}").is_in(yagofacts["subject"])) & 
        (~pl.col(f"index").is_in(tgt_indices))
    ).select(pl.col("index"))

    tgt_indices+=g1["index"].to_list()


In [215]:
len(tgt_indices)

2891

In [152]:
path_original=Path("/storage/store3/work/acvetkov/gitlab/KEN/experiments/datasets/movie_revenues/target.parquet")
df_target_og = pl.read_parquet(path_original)
df_target_og

raw_entities,col_to_embed,target
str,str,f64
"""Heat""","""<Heat_(1995_fi…",8.272855
"""Sudden Death""","""<Sudden_Death_…",7.80855
"""Cry, the Belov…","""<Cry,_the_Belo…",5.830284
"""Pocahontas""","""<Pocahontas_(1…",8.539176
"""Friday""","""<Friday_(1995_…",7.450494
"""Fair Game""","""<Fair_Game_(19…",7.061998
"""Bed of Roses""","""<Bed_of_Roses_…",7.279455
"""Screamers""","""<Screamers_(19…",6.762069
"""Black Sheep""","""<Black_Sheep_(…",1.50515


In [207]:
df_target_og.filter(
    ~pl.col("col_to_embed").is_in(yagofacts["subject"])
)

raw_entities,col_to_embed,target
str,str,f64
"""Calendar Girl""","""<Calendar_Girl…",6.409958
"""Batman""","""<Batman_(1989_…",8.61421
"""The Phantom""","""<The_Phantom_(…",7.238068
"""Dark City""","""<Dark_City_(19…",7.434574
"""Les Misérables…","""<Les_Misérable…",7.149106
"""Prom Night""","""<Prom_Night_(1…",7.170151
"""Lolita""","""<Lolita_(1997_…",6.025329
"""Atlantic City""","""<Atlantic_City…",7.104817
"""The General""","""<The_General_(…",6.08429


In [213]:
yagofacts.filter(yagofacts["subject"].str.contains("rrrrrrr"))

id,subject,predicate,cat_object,num_object,subject_formatted
str,str,str,str,f64,str


In [222]:
mapping_to_yago.filter(
    pl.col("title").is_in(df_target_og["raw_entities"])
)

title,release_date,revenue,title_format_1,title_format_2,title_format_3,index
str,str,str,str,str,str,i64
"""Toy Story""","""1995""","""373554033""","""<Toy_Story>""","""<Toy_Story_(fi…","""<Toy_Story_(19…",0
"""Jumanji""","""1995""","""262797249""","""<Jumanji>""","""<Jumanji_(film…","""<Jumanji_(1995…",1
"""Waiting to Exh…","""1995""","""81452156""","""<Waiting_to Ex…","""<Waiting_to Ex…","""<Waiting_to Ex…",2
"""Father of the …","""1995""","""76578911""","""<Father_of the…","""<Father_of the…","""<Father_of the…",3
"""Heat""","""1995""","""187436818""","""<Heat>""","""<Heat_(film)>""","""<Heat_(1995_fi…",4
"""Sudden Death""","""1995""","""64350171""","""<Sudden_Death>…","""<Sudden_Death_…","""<Sudden_Death_…",5
"""The American P…","""1995""","""107879496""","""<The_American …","""<The_American …","""<The_American …",7
"""Balto""","""1995""","""11348324""","""<Balto>""","""<Balto_(film)>…","""<Balto_(1995_f…",8
"""Nixon""","""1995""","""13681765""","""<Nixon>""","""<Nixon_(film)>…","""<Nixon_(1995_f…",9
